<a href="https://colab.research.google.com/github/karaage0703/code-cooker/blob/main/colab-notebooks/code_cooker_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Code Cooker

様々なLLMでデータ分析ができるノートブックです。


Simple data analysis notebook on google colab like Advanced Data Analysis(Code Interpreter).



## Basic setup

ライブラリのインストール

In [ ]:
!pip -qqq install japanize-matplotlib
!pip -qqq install qrcode

ライブラリのインポート

In [ ]:
import os
from google.colab import userdata
import random
import time
from IPython.core.magic import register_cell_magic
import matplotlib.pyplot as plt
import japanize_matplotlib

import io
from contextlib import redirect_stdout

システムプロンプトの設定

In [ ]:
system_prompt = f"""
    あなたはJupyterNotebookを使う優秀なPythonデータサイエンティストです。
    入力された指示をもとに、以下の要件に従ってデータ分析を実施してください。

    ###
    - 回答には、1つのPythonコードブロック（マジックコマンド、クラス定義、コンストラクタを含む）のみを含め、最初を```pythonで最後を```で囲ってください。
    - あなたは新しいPythonライブラリを自分で新たにインストールすることはできないので、ライブラリの不足時は、ユーザーにインストールする方法を提示してください。
    - 中身の分からないデータを操作するときは、まずはデータを確認するために読み込んだデータの情報(info, head)を出力するところまでだけを回答して。もう一度実行するようにユーザーにうながして下さい。
    - プロンプトにデータの情報が含まれていたら、その情報をもとに処理をすすめてください。
    """

## Select AI

Select AI type and execute AI setup code which you select.

In [ ]:
AI_TYPE = 'OPENAI' #@param ["OPENAI", "ANTHROPIC", "GEMINI", "PHI3", "LLAMA3"] {allow-input: true}

### OpenAI GPT-4o setup

In [ ]:
!pip -qqq install openai

from openai import OpenAI

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
client_openai = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

def chat_gpt(prompt_w_history):
    print("I am GPT")
    response = client_openai.chat.completions.create(
        model="gpt-4o-2024-08-06",
        messages=prompt_w_history,
    )

    return response.choices[0].message.content

### Anthropic Claude 3.5 Sonnet setup

In [ ]:
!pip -qqq install anthropic

import anthropic

os.environ["ANTHROPIC_API_KEY"] = userdata.get("ANTHROPIC_API_KEY")
client_anthropic = anthropic.Anthropic(api_key=os.getenv('ANTHROPIC_API_KEY'))

def chat_claude(prompt_w_history, system_prompt):
    print("I am Claude")
    response = client_anthropic.messages.create(
        max_tokens=2048,
        messages=prompt_w_history,
        model="claude-3-5-sonnet-20240620",
        system=system_prompt
    )

    return response.content[0].text

### Google Gemini 1.5 Pro setup

In [ ]:
!pip -qqq install google-generativeai

import google.generativeai as genai

os.environ["GEMINI_API_KEY"] = userdata.get("GEMINI_API_KEY")

genai.configure(
    api_key=os.getenv('GEMINI_API_KEY')
)

client_google = genai.GenerativeModel('gemini-1.5-pro-latest', system_instruction=[system_prompt])

def chat_gemini(prompt_w_history):
    print("I am Gemini")
    response = client_google.generate_content(
        prompt_w_history
    )

    return response.text

### Microsoft Phi-3 setup

In [ ]:
!pip install -qqq transformers accelerate

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    "microsoft/Phi-3-mini-128k-instruct"
)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-128k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)


def chat_phi3(prompt_w_history):
    print("I am Phi-3")

    prompt = tokenizer.apply_chat_template(prompt_w_history, tokenize=False, add_generation_prompt=True)

    # 推論の実行
    token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
    with torch.no_grad():
        output_ids = model.generate(
            token_ids.to(model.device),
            do_sample=True,
            temperature=0.6,
            max_new_tokens=256,
        )
    response = tokenizer.decode(output_ids.tolist()[0][token_ids.size(1) :], skip_special_tokens=True)

    return response

### Meta Llama3 setup

In [ ]:
!pip install -qqq transformers accelerate bitsandbytes

import torch
from huggingface_hub import login
from google.colab import userdata
from transformers import AutoModelForCausalLM, AutoTokenizer

login(token = userdata.get('HF_TOKEN'))

tokenizer = AutoTokenizer.from_pretrained(
    "meta-llama/Meta-Llama-3-8B-Instruct"
)
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Meta-Llama-3-8B-Instruct",
    device_map="auto",
    torch_dtype="auto",
)


def chat_llama3(prompt_w_history):
    print("I am Llama 3")

    prompt = tokenizer.apply_chat_template(prompt_w_history, tokenize=False, add_generation_prompt=True)

    token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
    with torch.no_grad():
        output_ids = model.generate(
            token_ids.to(model.device),
            do_sample=True,
            temperature=0.6,
            top_p=0.9,
            max_new_tokens=256,
            eos_token_id=[
                tokenizer.eos_token_id,
                tokenizer.convert_tokens_to_ids("<|eot_id|>")
            ],
        )
    response = tokenizer.decode(output_ids.tolist()[0][token_ids.size(1) :], skip_special_tokens=True)

    return response

## Code cook
関数の定義

In [ ]:
MAX_RETRAY_COUNT = 3

@register_cell_magic
def _(line, cell):
    global system_prompt

    user_prompt = cell
    filename = line

    if filename != "":
        system_prompt += f"""
        # 入力データ:ファイルパス
        {filename}
        なお、データはUTF-8で読み込んでください。
        """

    if AI_TYPE == "OPENAI":
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ]
    if AI_TYPE == "ANTHROPIC":
        messages = [
            {"role": "user", "content": user_prompt},
        ]
    if AI_TYPE == "GEMINI":
        messages = [
            {"role": "user", "parts": [user_prompt]},
        ]
    if AI_TYPE == "PHI3":
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
         ]
    if AI_TYPE == "LLAMA3":
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
         ]

    prompt_w_history.extend(messages)

    retry_count = 0
    while retry_count < MAX_RETRAY_COUNT:
        try:
            if AI_TYPE == "OPENAI":
                assistant_response = chat_gpt(prompt_w_history)
            if AI_TYPE == "ANTHROPIC":
                assistant_response = chat_claude(prompt_w_history, system_prompt)
            if AI_TYPE == "GEMINI":
                assistant_response = chat_gemini(prompt_w_history)
            if AI_TYPE == "PHI3":
                assistant_response = chat_phi3(prompt_w_history)
            if AI_TYPE == "LLAMA3":
                assistant_response = chat_llama3(prompt_w_history)

        except Exception as e:
            print(e)
            time.sleep(3)
            continue

        print("APIからの応答:", assistant_response)

        # Pythonコードの抽出
        python = ""
        if "```python" in assistant_response and "```" in assistant_response:
            python = assistant_response.split("```python")[1].split("```")[0]

        # print("抽出されたPythonコード:", python)

        try:
            stream = io.StringIO()

            with redirect_stdout(stream):
                exec(python)

            output_stream = stream.getvalue()

            print("Result:")
            print(output_stream)

            # プロンプトに追加
            if AI_TYPE == "GEMINI":
                prompt_w_history.append({"role": "model", "parts": [str(assistant_response)]})
            else:
                prompt_w_history.append({"role": "assistant", "content": str(assistant_response) + 'result:' + output_stream})
            break
        except Exception as e:
            # エラー処理
            print(f"エラーが発生しました: {e}")
            if AI_TYPE == "GEMINI":
                prompt_w_history.append({"role": "model", "parts": str(assistant_response)})
            else:
                prompt_w_history.append({"role": "assistant", "content": str(assistant_response)})
            retry_count += 1
            if retry_count < MAX_RETRAY_COUNT:
                # エラー内容をプロンプトに追加
                if AI_TYPE == "GEMINI":
                    prompt_w_history.append({"role": "user", "parts": [f"エラーが発生しました。エラーメッセージ: {e}\nコードを修正して再度実行してください。"]})
                else:
                    prompt_w_history.append({"role": "user", "content": f"エラーが発生しました。エラーメッセージ: {e}\nコードを修正して再度実行してください。"})

    print("コードの実行が完了しました。" if retry_count < MAX_RETRAY_COUNT else "コードの修正に失敗しました。再実行回数が上限に達しました。")

プロンプトの初期化

In [ ]:
prompt_w_history = []

## Example

In [ ]:
%%_
架空の国の地域ごとの人口比率のデータを可視化してください。日本語でお願いします。

In [ ]:
%%_
架空の国の地域ごとの人口比率のデータを可視化してください。データはないので適当でOKです。
日本語でお願いします。

In [ ]:
%%_ sample_excel.xlsx
アップロードしたExcelファイルの金額を月ごとに集計して棒グラフにしてください。

In [ ]:
%%_
以下のURLをQRコードにしてください。

https://karaage.hatenadiary.jp/

In [ ]:
%%_ soraumineko_01.jpg
画像をモノクロにして表示してください。

In [ ]:
%%_ face_sample.jpg
画像に対してdlibライブラリを使った顔検出をして表示してください。
「import dlib」でdlibライブラリを使用できます。

In [ ]:
%%_ newly_confirmed_cases_daily_japanese.csv
これは、新型コロナウイルスの全国と都道府県別での新規陽性者数のデータです。
このデータを可視化します。まず、全国の新規陽性者数の折れ線グラフを描いて表示してください。

In [ ]:
%%_ newly_confirmed_cases_daily_japanese.csv
東京都、大阪府、愛知県、福岡県の新規陽性者数について、折れ線グラフを描いて表示してください。
データは2022年1月以降のものを使ってください。

In [ ]:
%%_ newly_confirmed_cases_daily_japanese.csv
東京都、大阪府、愛知県、福岡県の新規陽性者数について、7日移動平均のグラフを描いて表示してください。
データは2022年1月以降のものを使ってください。

In [ ]:
%%_ newly_confirmed_cases_daily_japanese.csv
要件に従って、アニメーションGIFファイル 'anime.gif' を作成してください。

要件"""
・Pillowを使う。
・東京都、大阪府、愛知県、福岡県の新規陽性者について、7日移動平均のグラフを描く。
・全部で10コマのアニメーションにする。
・x軸を固定し、7日移動平均のグラフが徐々に右に伸びていくようにする。
・y軸を固定する。
"""

In [ ]:
import IPython

out_filename = 'anime.gif'
IPython.display.Image(out_filename, format='png')

## Reference:
- [claude_manim.ipynb](https://colab.research.google.com/github/wmoto-ai/claude-manim-notebook/blob/main/claude_manim.ipynb#scrollTo=mF8X8Q3LyZc9)
- [Google Colab で Llama 3 を試す](https://note.com/npaka/n/n73b0786f48e9)
- [Google Colab で Phi-3 を試す](https://note.com/npaka/n/n9915dbd44a84)
- [Google Colab上で手軽にAdvanced Data Analysis(Code Interpreter)的機能を実現する方法](https://zenn.dev/karaage0703/articles/f6a1df0b2eabf4)
- [LLMに面倒なことをやらせるソフト「Code Cooker」の紹介](https://zenn.dev/karaage0703/articles/8996f211fc38cd)